입력 데이터:  
다운로드: https://github.com/nongaussian/class-2020-spark/blob/master/Week%207/input.zip  
설명 - 영화DB사이트 (MovieLens)에서 수집된 데이터로서 u.data에는 사용자가 영화에 매긴 평점이 담겨있고  
u.item에는 영화마다 제목, 카테고리 등의 정보가 담겨 있다.  
  
입력파일 1 - u.data  

* 라인마다 사용자의 영화평점이 기록되어 있으며 4개의 속성 "user id \t item id \t rating \t timestamp"가 탭으로 구분되어 작성되어 있다.  
* (즉, split("\t") 로 자를 수 있음)  모두 integer값이다.  
* 한 사람이 여러개의 영화에 평점을 매길 수 있으므로 같은 user id로 시작하는 라인이 복수번 나타날수 있다.  
* item id는 아래 u.item파일에 있는 영화 아이디 (movie id)를 가리킨다 (즉, foreign key)  
* rating은 1에서 5까지 범위이다.  
  
입력파일 2 - u.item  
  
* 라인마다 영화 정보가 기록되어 있으며 "movie id | movie title | release date | video release date |
* IMDb URL | unknown | Action | Adventure | Animation |
* Children's | Comedy | Crime | Documentary | Drama | Fantasy |
* Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
* Thriller | War | Western" 의 속성 들이 파이프라인(|)으로 구분되어 작성되어 있다.
* 그러나 아래 문제를 보면 맨 앞의 두개 속성, movie id와 movie title 만 사용한다.
* movie id는 이 파일 안에서 한번만 나타난다 (즉, primary key)
  
문제.  
5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수는?  
'T'는 대문자  
주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오.   
즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  


In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "transformation 3")

In [2]:
# 공백으로 구분하여 item id (movie id)와 rating만 슬라이싱하여 data로 추출한다 
data = sc.textFile('u.data').map(lambda x: x.split("\t")[1:3])
# data.collect()
# [['242', '3'],
#  ['302', '3'],
#  ['377', '1'],
#  ['51', '2'],
#  ['346', '1'],
#  ['474', '4'],,,,]

In [3]:
# |로 구분하여  movie id 와 movie title만 슬라이싱하여 item으로 추출한다 # 중복값 제거
# item = sc.textFile('u.item').map(lambda x: x.split("|")[0:2]).distinct() # error
item = sc.textFile('u.item').map(lambda x: x.split("|")[0:2])
item.distinct()
# item.collect()
# [['1', 'Toy Story (1995)'],
#  ['2', 'GoldenEye (1995)'],
#  ['3', 'Four Rooms (1995)'],
#  ['4', 'Get Shorty (1995)'],
#  ['5', 'Copycat (1995)'],,,]

PythonRDD[8] at RDD at PythonRDD.scala:53

In [4]:
# cogroup 이용
# groupByKey()는 동일 rdd내에서, cogroup은 다른 2개의 RDD의 key값을 중심으로 (key, tuple(lterable, lterable,,))
rdd = data.cogroup(item).mapValues(lambda t: ( list(t[0]), list(t[1]) ))
# [('346',
#   (['1',
#     '4',
#     '3',
#     '4',
#     '3',
#     '4',
#     '1',
#     ,,,,,,,,,,,,
#    '5',
#     '4',
#     '2',
#     '5'],
#    ['Terminator 2: Judgment Day (1991)'])),
#  ('307',
#   (['4',
#     '4',

In [ ]:
# move id만
rdd.map(lambda x: x[0]).collect()
# ['346',
#  '474',
#  '265',
#  '465',
#  '451',
#  '86',
#  '257',
#  '222',
#  '40',,,]

In [ ]:
# rdd중에서 중복된 값이 있는지 확인 # 없다
rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] != 1).count()
# 0

In [ ]:
# rating, movie title
rdd.map(lambda x: x[1]).collect()
# [(['1',
#    '4',
#    '3',
#    '4',
#    '3',
#    '4',,,,,,,,
#    '1',
#    '1',
#    '5',
#    '5',
#    '4'],
#   ['Jackie Brown (1997)']),
#      ,
#     ,
#     ,

In [ ]:
rdd.map(lambda x: x[1][0]).collect()
# [['1',
#   '4',
#   '3',
#   '4',
#   '3',
#   '4',
#   '1',
#   '4',
rdd.map(lambda x: x[1][1]).collect() 
# [['Jackie Brown (1997)'],
#  ['Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)'],
#  ['Hunt for Red October, The (1990)'],
#  ['Jungle Book, The (1994)'],
#  ['Grease (1978)'],
#  ['Remains of the Day, The (1993)'],,,,

In [ ]:
# rdd.map(lambda x : x[1][0].contains['5']).collect() # error
# rdd.map(lambda x: x[1][0]).filter(lambda y : y =='5' ).collect() # error
# rdd.map(lambda x: x[1][0]).filter(lambda y : y.contains['5'] ).collect() # error
# rdd.filter(lambda x : x[1][0].contains['5'] ).collect() # error
# rdd.filter(lambda x : x[1][0] in '5' ).collect() # error
# rdd.filter(lambda x : x[1][1][0:2] in 'T' ).collect() # error

rdd.filter(lambda x : '5' in x[1][0]).collect() 
# [('346',
#   (['1',
#     '4',
#     '3',
#     '4',
#     '3',
#     '4',
#     '1',
#     '4',
#     '4',
# rdd.filter(lambda x : '5' in x[1][0] and 'T' in x[1][1]).collect() # []

In [ ]:
rdd.map(lambda x : x[1][1][0][0:1]).collect() # []
# ['J',
#  'D',
#  'H',
#  'J',
#  'G',
#  'R',


In [ ]:
rdd.filter(lambda x : '5' in x[1][0] and x[1][1][0][0:1] == 'T').collect()
# [('40',
#   (['3',
#     '3',
#     '3',
#     '3',
#     '1',
#     '1',
#     '4',

In [5]:
type(rdd.filter(lambda x : '5' in x[1][0] and x[1][1][0][0:1] == 'T').collect()) # list
len(rdd.filter(lambda x : '5' in x[1][0] and x[1][1][0][0:1] == 'T').collect())

75